# Top2Vec Hyperparameter Tuning

Based on `coherence_results_v1.csv`, `sentence-transformers/all-MiniLM-L6-v2` achieved the highest average coherence across all subjects. This notebook performs grid-search hyperparameter tuning over UMAP and HDBSCAN parameters to find the best Top2Vec configuration.

In [1]:
import os
import gc
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List, Optional, Dict, Any
from tqdm import tqdm
from itertools import product
import warnings
import time

from top2vec import Top2Vec
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel

warnings.filterwarnings("ignore", category=FutureWarning)

## Configuration

In [2]:
VERSION = "v1"
LIST_SUBJECT = ["cs", "math", "physics"]

TRANSFORMER = "sentence-transformers/all-MiniLM-L6-v2"
EMBEDDING_DIM = 384

BASE_DIR = Path("../../dataset")
EMBEDDING_DIR = Path("../bertopic/embedding")
TUNNING_DIR = Path("./tunning")

SAFE_MODEL_NAME = TRANSFORMER.replace("/", "_").replace("-", "_")
OUTPUT_DIR = TUNNING_DIR / SAFE_MODEL_NAME
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Embedding: {TRANSFORMER}")
print(f"Output directory: {OUTPUT_DIR}")

Embedding: sentence-transformers/all-MiniLM-L6-v2
Output directory: tunning/sentence_transformers_all_MiniLM_L6_v2


## Hyperparameter Grid

In [3]:
PARAM_GRID = {
    "umap_n_neighbors": [10, 15, 30],
    "umap_n_components": [5, 10, 30],
    "hdbscan_min_cluster_size": [15, 30, 50],
    "hdbscan_cluster_selection_method": ["eom"],
    "min_count": [50],
}

keys = list(PARAM_GRID.keys())
values = list(PARAM_GRID.values())
all_combos = list(product(*values))

print(f"Total parameter combinations: {len(all_combos)}")
print(f"Total runs (combinations x subjects): {len(all_combos) * len(LIST_SUBJECT)}")

Total parameter combinations: 27
Total runs (combinations x subjects): 81


## Helper Functions

In [4]:
def get_model_safe_name(model_name: str) -> str:
    return model_name.replace("/", "_").replace("-", "_")


def load_dataset(subject: str) -> Optional[pd.DataFrame]:
    file_path = BASE_DIR / subject / "emb" / f"{VERSION}.csv"
    if not file_path.exists():
        print(f"File not found: {file_path}")
        return None
    return pd.read_csv(file_path)


def load_mmap_embeddings(
    mmap_path: str,
    num_documents: int,
    embedding_dim: int,
    dtype: str = "float32"
) -> Optional[np.ndarray]:
    try:
        embs = np.array(np.memmap(
            mmap_path, dtype=dtype, mode="r",
            shape=(num_documents, embedding_dim)
        ))
        return normalize(embs)
    except FileNotFoundError:
        print(f"Embedding not found: {mmap_path}")
        return None
    except Exception as e:
        print(f"Error loading embeddings: {e}")
        return None


def train_top2vec_with_precomputed(
    documents: List[str],
    precomputed_embeddings: np.ndarray,
    transformer_name: str,
    umap_args: Dict[str, Any] = None,
    hdbscan_args: Dict[str, Any] = None,
    min_count: int = 50,
) -> Top2Vec:
    num_docs = len(documents)
    st_model = SentenceTransformer(transformer_name)

    original_embed_docs = Top2Vec._embed_documents

    def patched_embed_documents(self, train_corpus, batch_size):
        if len(train_corpus) == num_docs:
            return precomputed_embeddings
        else:
            return st_model.encode(train_corpus, batch_size=batch_size, show_progress_bar=False)

    Top2Vec._embed_documents = patched_embed_documents

    model = Top2Vec(
        documents=documents,
        embedding_model='all-MiniLM-L6-v2',
        min_count=min_count,
        contextual_top2vec=False,
        ngram_vocab=False,
        umap_args=umap_args,
        hdbscan_args=hdbscan_args,
        verbose=False,
    )

    Top2Vec._embed_documents = original_embed_docs
    del st_model

    return model


def calculate_coherence(
    model: Top2Vec,
    texts_tokenized: List[List[str]],
    dictionary: Dictionary,
    top_n: int = 5
) -> float:
    num_topics = model.get_num_topics()
    topic_words, _, _ = model.get_topics(num_topics)
    topic_words_sliced = topic_words[:, :top_n]

    cm = CoherenceModel(
        topics=topic_words_sliced.tolist(),
        texts=texts_tokenized,
        dictionary=dictionary,
        coherence='c_v',
        processes=1
    )

    return cm.get_coherence()

## Load Datasets, Embeddings & Tokenize

In [5]:
all_data = {}
all_embeddings = {}
all_texts_tokenized = {}
all_dictionaries = {}

safe_name = get_model_safe_name(TRANSFORMER)

for subject in LIST_SUBJECT:
    df = load_dataset(subject)
    if df is None:
        continue

    all_data[subject] = df
    print(f"{subject}: {len(df):,} documents loaded")

    mmap_path = EMBEDDING_DIR / subject / f"{safe_name}_{VERSION}.mmap"
    embs = load_mmap_embeddings(str(mmap_path), len(df), EMBEDDING_DIM)
    if embs is None:
        print(f"  ⚠ Skipping {subject}: embedding not found")
        continue
    all_embeddings[subject] = embs
    print(f"  Embeddings loaded: {embs.shape}")

    print(f"  Tokenizing for coherence...")
    texts_tokenized = [text.split() for text in tqdm(df['text'].fillna('').tolist(), desc=f"  {subject}")]
    all_texts_tokenized[subject] = texts_tokenized
    all_dictionaries[subject] = Dictionary(texts_tokenized)

print(f"\nSubjects ready: {list(all_embeddings.keys())}")

cs: 165,756 documents loaded
  Embeddings loaded: (165756, 384)
  Tokenizing for coherence...


  cs: 100%|██████████| 165756/165756 [00:02<00:00, 72310.43it/s]


math: 157,085 documents loaded
  Embeddings loaded: (157085, 384)
  Tokenizing for coherence...


  math: 100%|██████████| 157085/157085 [00:01<00:00, 134250.09it/s]


physics: 146,311 documents loaded
  Embeddings loaded: (146311, 384)
  Tokenizing for coherence...


  physics: 100%|██████████| 146311/146311 [00:02<00:00, 63082.18it/s]



Subjects ready: ['cs', 'math', 'physics']


## Hyperparameter Tuning Grid Search

In [6]:
results = []
csv_path = OUTPUT_DIR / "tuning_results.csv"
best_coherence = {subject: -1.0 for subject in all_embeddings}

total_runs = len(all_combos) * len(all_embeddings)
run_count = 0

for subject in all_embeddings:
    df = all_data[subject]
    documents = df["text"].fillna("").tolist()
    embs = all_embeddings[subject]

    print(f"{"=" * 70}")
    print(f"Subject: {subject.upper()} ({len(documents):,} documents)")
    print(f"{"=" * 70}")

    for combo in all_combos:
        run_count += 1
        params = dict(zip(keys, combo))

        umap_args = {
            "n_neighbors": params["umap_n_neighbors"],
            "n_components": params["umap_n_components"],
            "metric": "cosine",
        }
        hdbscan_args = {
            "min_cluster_size": params["hdbscan_min_cluster_size"],
            "metric": "euclidean",
            "cluster_selection_method": params["hdbscan_cluster_selection_method"],
        }

        print(f"[{run_count}/{total_runs}] {subject} | "
              f"nn={params["umap_n_neighbors"]} nc={params["umap_n_components"]} "
              f"mcs={params["hdbscan_min_cluster_size"]} csm={params["hdbscan_cluster_selection_method"]} "
              f"mc={params["min_count"]}")

        try:
            start_time = time.time()

            model = train_top2vec_with_precomputed(
                documents=documents,
                precomputed_embeddings=embs,
                transformer_name=TRANSFORMER,
                umap_args=umap_args,
                hdbscan_args=hdbscan_args,
                min_count=params["min_count"],
            )

            n_topics = model.get_num_topics()
            elapsed = time.time() - start_time

            if n_topics <= 1:
                print(f"  ⚠ Only {n_topics} topic(s) found, skipping coherence ({elapsed:.1f}s)")
                coherence = None
            else:
                coherence = calculate_coherence(
                    model,
                    all_texts_tokenized[subject],
                    all_dictionaries[subject]
                )
                print(f"  ✓ Topics: {n_topics} | Coherence: {coherence:.4f} ({elapsed:.1f}s)")

                if coherence > best_coherence[subject]:
                    best_coherence[subject] = coherence
                    save_path = OUTPUT_DIR / f"best_model_{subject}"
                    save_path.mkdir(parents=True, exist_ok=True)
                    model.save(str(save_path / "model"))
                    print(f"  🏆 New best for {subject}! Coherence: {coherence:.4f} → Model saved to {save_path}")

            result_row = {
                "subject": subject,
                "umap_n_neighbors": params["umap_n_neighbors"],
                "umap_n_components": params["umap_n_components"],
                "hdbscan_min_cluster_size": params["hdbscan_min_cluster_size"],
                "hdbscan_cluster_selection_method": params["hdbscan_cluster_selection_method"],
                "min_count": params["min_count"],
                "n_topics": n_topics,
                "coherence": coherence,
                "time_seconds": round(elapsed, 1),
            }
            results.append(result_row)

            del model
            gc.collect()

        except Exception as e:
            print(f"  ✗ Error: {e}")
            result_row = {
                "subject": subject,
                "umap_n_neighbors": params["umap_n_neighbors"],
                "umap_n_components": params["umap_n_components"],
                "hdbscan_min_cluster_size": params["hdbscan_min_cluster_size"],
                "hdbscan_cluster_selection_method": params["hdbscan_cluster_selection_method"],
                "min_count": params["min_count"],
                "n_topics": None,
                "coherence": None,
                "time_seconds": None,
            }
            results.append(result_row)

        if run_count % 10 == 0:
            pd.DataFrame(results).to_csv(csv_path, index=False)
            print(f"  💾 Checkpoint saved ({run_count}/{total_runs})")

results_df = pd.DataFrame(results)
results_df.to_csv(csv_path, index=False)
print(f"✅ All results saved to {csv_path}")
print(f"Total runs: {len(results_df)}")

Subject: CS (165,756 documents)
[1/81] cs | nn=10 nc=5 mcs=15 csm=eom mc=50


/home/nedo/Kuliah/TA/Program/.venv/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2026-02-15 01:03:28,139 - top2vec - INFO - Downloading all-MiniLM-L6-v2 model


  ✓ Topics: 914 | Coherence: 0.5700 (82.2s)
  🏆 New best for cs! Coherence: 0.5700 → Model saved to tunning/sentence_transformers_all_MiniLM_L6_v2/best_model_cs
[2/81] cs | nn=10 nc=5 mcs=30 csm=eom mc=50


/home/nedo/Kuliah/TA/Program/.venv/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2026-02-15 01:07:00,148 - top2vec - INFO - Downloading all-MiniLM-L6-v2 model


  ✓ Topics: 505 | Coherence: 0.5748 (65.1s)
  🏆 New best for cs! Coherence: 0.5748 → Model saved to tunning/sentence_transformers_all_MiniLM_L6_v2/best_model_cs
[3/81] cs | nn=10 nc=5 mcs=50 csm=eom mc=50


...

  ✓ Topics: 579 | Coherence: 0.6281 (78.9s)
[80/81] physics | nn=30 nc=30 mcs=30 csm=eom mc=50


/home/nedo/Kuliah/TA/Program/.venv/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2026-02-15 04:04:52,318 - top2vec - INFO - Downloading all-MiniLM-L6-v2 model


  ✓ Topics: 304 | Coherence: 0.6431 (80.0s)
  💾 Checkpoint saved (80/81)
[81/81] physics | nn=30 nc=30 mcs=50 csm=eom mc=50


/home/nedo/Kuliah/TA/Program/.venv/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2026-02-15 04:07:15,508 - top2vec - INFO - Downloading all-MiniLM-L6-v2 model


  ✓ Topics: 207 | Coherence: 0.6623 (80.3s)
  🏆 New best for physics! Coherence: 0.6623 → Model saved to tunning/sentence_transformers_all_MiniLM_L6_v2/best_model_physics
✅ All results saved to tunning/sentence_transformers_all_MiniLM_L6_v2/tuning_results.csv
Total runs: 81


## Results Summary

In [7]:
results_df = pd.read_csv(csv_path)
valid_results = results_df.dropna(subset=["coherence"])

print(f"Total runs: {len(results_df)}")
print(f"Valid runs (>1 topic): {len(valid_results)}")
print(f"Skipped (1 topic or error): {len(results_df) - len(valid_results)}")

print("\n" + "=" * 90)
print("Best Parameters per Subject")
print("=" * 90)

best_per_subject = {}
for subject in LIST_SUBJECT:
    subj_results = valid_results[valid_results["subject"] == subject]
    if subj_results.empty:
        print(f"\n{subject.upper()}: No valid results")
        continue

    best_idx = subj_results["coherence"].idxmax()
    best_row = subj_results.loc[best_idx]
    best_per_subject[subject] = best_row

    print(f"\n{subject.upper()}:")
    print(f"  Best coherence: {best_row['coherence']:.4f}")
    print(f"  Topics: {int(best_row['n_topics'])}")
    print(f"  umap_n_neighbors: {int(best_row['umap_n_neighbors'])}")
    print(f"  umap_n_components: {int(best_row['umap_n_components'])}")
    print(f"  hdbscan_min_cluster_size: {int(best_row['hdbscan_min_cluster_size'])}")
    print(f"  hdbscan_cluster_selection_method: {best_row['hdbscan_cluster_selection_method']}")
    print(f"  min_count: {int(best_row['min_count'])}")

print("\n" + "=" * 90)
print("Top 5 per Subject")
print("=" * 90)
for subject in LIST_SUBJECT:
    subj_results = valid_results[valid_results["subject"] == subject]
    if subj_results.empty:
        continue
    top5 = subj_results.nlargest(5, "coherence")
    print(f"\n{subject.upper()}:")
    print(top5[["umap_n_neighbors", "umap_n_components", "hdbscan_min_cluster_size",
                "hdbscan_cluster_selection_method", "min_count", "n_topics", "coherence"]].to_string(index=False))

Total runs: 81
Valid runs (>1 topic): 81
Skipped (1 topic or error): 0

Best Parameters per Subject

CS:
  Best coherence: 0.5934
  Topics: 248
  umap_n_neighbors: 30
  umap_n_components: 10
  hdbscan_min_cluster_size: 50
  hdbscan_cluster_selection_method: eom
  min_count: 50

MATH:
  Best coherence: 0.5651
  Topics: 198
  umap_n_neighbors: 30
  umap_n_components: 5
  hdbscan_min_cluster_size: 50
  hdbscan_cluster_selection_method: eom
  min_count: 50

PHYSICS:
  Best coherence: 0.6623
  Topics: 207
  umap_n_neighbors: 30
  umap_n_components: 30
  hdbscan_min_cluster_size: 50
  hdbscan_cluster_selection_method: eom
  min_count: 50

Top 5 per Subject

CS:
 umap_n_neighbors  umap_n_components  hdbscan_min_cluster_size hdbscan_cluster_selection_method  min_count  n_topics  coherence
               30                 10                        50                              eom         50       248   0.593423
               15                 10                        50                  

## Load Saved Models & Show Coherence

In [8]:
for subject in LIST_SUBJECT:
    model_path = OUTPUT_DIR / f"best_model_{subject}" / "model"
    if not model_path.exists():
        print(f"{subject.upper()}: No saved model found at {model_path}")
        continue

    model = Top2Vec.load(str(model_path))
    n_topics = model.get_num_topics()

    coherence = calculate_coherence(
        model,
        all_texts_tokenized[subject],
        all_dictionaries[subject]
    )
    print(f"{subject.upper()}: Topics={n_topics} | Coherence={coherence:.4f}")

    del model
    gc.collect()


CS: Topics=248 | Coherence=0.5934
MATH: Topics=198 | Coherence=0.5651
PHYSICS: Topics=207 | Coherence=0.6623
